# 102 LangGraph: Core Concepts

**Workshop**: LangGraph 101
**Duration**: ~30 minutes
**Difficulty**: Beginner

## Learning Objectives

By completing this notebook, you will:
- Understand State and how it flows through LangGraph applications
- Learn to create Nodes that process and update state
- Master Graph construction with StateGraph
- Use Edges to define workflow connections
- Implement Conditional Edges for dynamic routing
- Integrate Tools and ToolNode for external capabilities
- Build a complete network automation workflow

## Prerequisites

- **Knowledge**: Completed 101 notebook (type annotations), basic Python
- **Setup**: None required for this conceptual notebook

## Table of Contents

1. [Introduction](#1-introduction)
2. [State - The Foundation](#2-state---the-foundation)
3. [Nodes - Processing Units](#3-nodes---processing-units)
4. [Graph and Edges](#4-graph-and-edges)
5. [What's Next: Conditional Routing](#5-whats-next-conditional-routing)
6. [What's Next: Tools and External Capabilities](#6-whats-next-tools-and-external-capabilities)
7. [Messages in LangGraph](#7-messages-in-langgraph)
8. [Hands-On Exercises](#8-hands-on-exercises)
9. [Summary](#9-summary)

## 1. Introduction

LangGraph is a framework for building stateful, multi-actor applications with Large Language Models (LLMs). At its core, LangGraph provides a graph-based approach to orchestrating workflows where data flows through nodes connected by edges. Think of it like a network diagram - but instead of routers and switches, you have processing nodes that transform state as it moves through your application.

In this notebook, we'll explore the fundamental building blocks of LangGraph without worrying about LLMs or APIs. We'll focus on understanding the core concepts using simple network automation examples.

### Why LangGraph Matters for Network Automation

Without a structured workflow framework:
- Complex automation logic becomes spaghetti code with nested if/else statements
- State management is error-prone and hard to debug
- Adding new steps or changing workflow order requires significant refactoring

With LangGraph:
- Workflows are visual and easy to understand (nodes and edges)
- State is explicitly defined and flows predictably through the graph
- Adding, removing, or reordering steps is as simple as modifying the graph structure

### What We'll Build

In this notebook, we'll learn:
1. How to define **State** using TypedDict (building on 101 concepts)
2. How to create **Nodes** that process state
3. How to construct a **Graph** and connect nodes with **Edges**
4. How to add **Conditional Edges** for dynamic routing
5. How to integrate **Tools** for external capabilities
6. How all these pieces come together in a complete workflow

Let's get started!

### 1.1 Import Core LangGraph Components

Let's import the essential LangGraph components we'll be working with, including visualization tools:

In [ ]:
# Core typing imports
from typing import TypedDict, Annotated, Literal
from pprint import pprint

# LangGraph core components
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

# Tool decorator
from langchain_core.tools import tool

# Visualization imports
from IPython.display import Image, display

print("✅ Imports successful")
print("\nCore components loaded:")
print("  - TypedDict: For defining state schemas")
print("  - StateGraph: For building graphs")
print("  - START/END: Special graph entry/exit points")
print("  - ToolNode: For integrating external tools")
print("  - tool: Decorator for creating tools")
print("  - IPython.display: For visualizing graphs")

---

## 2. State - The Foundation

### What is State?

**State** is a shared data structure that holds the current information or context of your entire LangGraph application. In simpler terms, it's like your application's memory where it keeps track of variables and data that nodes can access and modify as they execute.

Think of state like the information that needs to be tracked during **SCM address object creation**. When a network administrator creates address objects in Strata Cloud Manager, they need to track multiple pieces of information throughout the process:
- The address object name
- IP address or FQDN value
- Which folder to deploy to
- Description and tags
- Validation status
- Whether the object was created successfully
- Any errors encountered
- Current step in the workflow

This information needs to persist and be accessible throughout the entire configuration workflow. Each task in the workflow (parsing input, validating data, creating the object in SCM) can read this information and update it. That's exactly what **State** does in LangGraph - it's the shared data that flows through your workflow, being read and updated by each step (node).

### Key Points About State

- **Shared Data Structure**: All nodes can access and modify the same state
- **Application Memory**: Keeps track of variables and data throughout execution
- **Type-Safe**: Using TypedDict ensures you know exactly what data exists
- **Centralized**: All information your workflow needs is stored in one place
- **Flows Through Workflow**: Each step receives state, processes it, and returns updates

### Why State Matters

State is the backbone of your LangGraph application. Without properly defined state:
- Each step doesn't know what information is available from previous steps
- Type errors happen at runtime instead of development time
- Debugging becomes difficult because data flow is unclear

With well-defined state:
- Every step knows exactly what data is available
- IDEs provide autocomplete for state fields
- Data flow is explicit and easy to trace
- You can see the complete status of your workflow at any point

### Progressive Complexity in State Design

As you build more sophisticated workflows, your state schemas will evolve. Here's how state complexity typically progresses:

| Level | Complexity | Fields | Example Use Case |
|-------|-----------|--------|------------------|
| **Level 1** | Simple (2-3 fields) | `name`, `ip_netmask`, `folder` | Basic address object creation |
| **Level 2** | Moderate (4-6 fields) | Add: `validated`, `error_message`, `created` | With validation and tracking |
| **Level 3** | Complex (7-10 fields) | Add: `description`, `tags`, `object_id`, `current_step` | Full lifecycle tracking |
| **Level 4** | Advanced (10+ fields) | Add: `api_response`, `errors` (list), `metadata`, `timestamps` | Production-ready with audit trail |

**Recommendation**: Start at Level 1-2 for learning, move to Level 3-4 for production workflows.

### 2.1 Defining State with TypedDict

In [ ]:
# Define state for an SCM address object creation workflow
from typing import Optional

class SCMAddressState(TypedDict):
    """State for SCM address object creation and management workflow."""
    # Input data
    name: str
    address_type: str  # "ip_netmask", "fqdn", or "ip_range"
    address_value: str  # IP/mask, FQDN, or IP range
    folder: str
    description: Optional[str]
    tags: list[str]
    
    # Workflow tracking
    validated: bool
    created: bool
    object_id: Optional[str]
    current_step: str
    errors: list[str]
    
    # API response data
    api_response: Optional[dict]

print("✅ SCMAddressState defined")
print("\nState schema for SCM address object workflow:")
for field_name, field_type in SCMAddressState.__annotations__.items():
    # Handle generic types like list[str] and Optional
    type_str = str(field_type).replace("typing.", "")
    print(f"  - {field_name}: {type_str}")

### 2.1b Modern State Patterns with Annotated Types

LangGraph supports modern Python type annotations using `Annotated` types with **reducers**. Reducers are functions that determine how state updates are merged when multiple nodes update the same field.

**Why Use Annotated Types?**

Without reducers, you have to manually merge lists and handle state updates:
```python
# Old way - manual merging
errors = state.get('errors', []) + ["New error"]
return {"errors": errors}
```

With reducers, LangGraph automatically handles merging:
```python
# New way - automatic merging
return {"errors": ["New error"]}  # Automatically appended!
```

**Common Reducers:**
- `operator.add`: Concatenate lists or add numbers
- Custom functions: Define your own merge logic

In [ ]:
# Modern state schema using Annotated types with reducers
from operator import add

class ModernSCMAddressState(TypedDict):
    """Modern state schema using Annotated types with reducers.
    
    Annotated[list[str], add] automatically merges lists when nodes
    return updates, eliminating manual concatenation.
    """
    # Input data
    name: str
    address_type: str
    address_value: str
    folder: str
    description: Optional[str]
    tags: Annotated[list[str], add]  # Auto-merge with add reducer
    
    # Workflow tracking
    validated: bool
    created: bool
    object_id: Optional[str]
    current_step: str
    errors: Annotated[list[str], add]  # Auto-merge errors from multiple nodes
    
    # API response data
    api_response: Optional[dict]

print("✅ ModernSCMAddressState defined with Annotated types")
print("\n💡 Benefits of Annotated Types with Reducers:")
print("   - No manual list concatenation needed")
print("   - Multiple nodes can append to same list field")
print("   - LangGraph handles merging automatically")
print("\nExample:")
print("  Old way: return {'errors': state.get('errors', []) + ['New error']}")
print("  New way: return {'errors': ['New error']}  # Auto-merged!")
print("\n  With add reducer, both nodes' errors are combined automatically:")

### 2.2 Starting Simple: Progressive State Complexity

When learning LangGraph, it's helpful to start with simpler state schemas and add complexity as needed. Let's see how the same address object workflow can be built with increasing complexity:

**Level 1 - Minimal State (3-4 fields)**

In [ ]:
# Simplest possible address object state
class SimpleAddressState(TypedDict):
    """Minimal state for basic address object creation."""
    name: str
    ip_netmask: str
    folder: str

print("✅ SimpleAddressState: Just 3 required fields")
print("   Perfect for learning the basics!")

**Level 2 - Add Validation (5-6 fields)**

In [ ]:
# Add validation tracking
class ValidatedAddressState(TypedDict):
    """State with validation tracking."""
    name: str
    ip_netmask: str
    folder: str
    validated: bool
    error_message: str

print("✅ ValidatedAddressState: Added validation tracking")
print("   Now we can track if the address object is valid!")

**Level 3 - Full Workflow (10+ fields)**

The `SCMAddressState` we defined above represents Level 3 - a production-ready state schema with all the fields needed for a complete workflow including creation tracking, API responses, and error handling.

**When to use each level:**
- **Level 1**: Learning exercises, simple demos
- **Level 2**: Workflows with validation but minimal tracking
- **Level 3**: Production workflows with full lifecycle management

💡 **Best Practice**: Start simple and add fields as your workflow needs grow!

### 2.3 How State Gets Updated Throughout the Workflow

In LangGraph, each step (node) in your workflow receives the current state, processes it, and returns a dictionary with the fields it wants to update. LangGraph then merges these updates into the state. This is how information flows through your upgrade workflow - each task updates only the parts relevant to it.

In [ ]:
# Example: Initial state when SCM workflow starts
initial_state: SCMAddressState = {
    "name": "web-server-pool",
    "address_type": "ip_netmask",
    "address_value": "10.1.1.0/24",
    "folder": "Texas",
    "description": "Web server subnet",
    "tags": ["Production", "WebServers"],
    "validated": False,
    "created": False,
    "object_id": None,
    "current_step": "starting",
    "errors": [],
    "api_response": None
}

print("Initial State (start of SCM address creation workflow):")
pprint(initial_state)

# Example: After the "validate_address" step completes
# This step only updates validation status and current_step
validation_update = {
    "validated": True,
    "current_step": "validated"
}

print("\nUpdate from 'validate_address' step:")
pprint(validation_update)

# LangGraph merges this update into the state
state_after_validation = {**initial_state, **validation_update}

print("\nState after validation:")
pprint(state_after_validation)

# Example: After the "create_in_scm" step completes
creation_update = {
    "created": True,
    "object_id": "550e8400-e29b-41d4-a716-446655440000",
    "current_step": "created",
    "api_response": {
        "name": "web-server-pool",
        "ip_netmask": "10.1.1.0/24",
        "folder": "Texas"
    }
}

print("\nUpdate from 'create_in_scm' step:")
pprint(creation_update)

# Merge again
state_after_creation = {**state_after_validation, **creation_update}

print("\nState after SCM creation:")
pprint(state_after_creation)

print("\n💡 Key Insight: State persists throughout the workflow!")
print("   - Validation status from step 1 is still available in step 2")
print("   - Each step adds information without losing previous data")
print("   - Later steps can see results from all earlier steps")
print("   - This is how we track the complete lifecycle of SCM objects!")

---

## 3. Nodes - Processing Units

### What is a Node?

**Nodes** are individual functions or operations that perform specific tasks within your LangGraph workflow. Each node receives an input (typically the current state), processes it, and produces an output (an updated state).

Think of nodes like **stages in an SCM configuration workflow**. When you create address objects in Strata Cloud Manager, you follow a series of steps:
1. **Parse Station**: Extracts address object details from user input
2. **Validation Station**: Validates IP formats, required fields, and folder existence
3. **Preparation Station**: Formats data according to SCM API requirements
4. **Creation Station**: Makes the API call to create the object in SCM
5. **Verification Station**: Confirms the object was created and retrieves its ID

Each station does one specific job. The configuration data moves through each station, and at each stage something specific happens to it. That's exactly what nodes are - each node performs one specific task in your workflow.

### Key Points About Nodes

- **Individual Functions**: Each node is a Python function with a single responsibility
- **Receives State**: Nodes get the current state as input
- **Processes Data**: Performs a specific operation (validation, API call, decision logic)
- **Returns Updates**: Returns a dictionary with state updates
- **Sequential or Parallel**: Nodes can run one after another or in parallel

### Why Nodes Matter

Nodes are where the actual work happens in your LangGraph application:
- Each node has a clear, single responsibility (like validating input or calling the SCM API)
- Breaking workflows into nodes makes them easier to test and debug
- You can reuse nodes across different workflows
- Nodes can be swapped, added, or removed without affecting others

### 3.1 Creating a Simple Node Function

In [ ]:
def validate_address_input(state: SCMAddressState) -> dict:
    """
    Node: Validate address object input data.
    
    This is the first station in our SCM workflow.
    It validates that all required fields are present and correctly formatted.
    
    Production Pattern:
        See docs/examples/address_objects.py:
        - Lines 9-15: IP/netmask configuration structure
        - Lines 21-26: FQDN configuration structure
        - Lines 32-37: IP range configuration structure
        
        Validation would check that input matches one of these patterns
        before attempting to create the object in SCM.
    
    Args:
        state: Current workflow state
        
    Returns:
        Dictionary with validation status updates
    """
    import re
    
    print(f"🔍 Validating address object: {state['name']}...")
    
    # In a real implementation, this would perform comprehensive validation
    # For this example, we'll simulate validation checks
    
    errors = []
    
    # Validate name
    if not state['name'] or len(state['name']) < 1:
        errors.append("Name is required")
    
    # Validate address type and value
    if state['address_type'] == "ip_netmask":
        # Check IP/netmask format
        ip_pattern = r'^(\d{1,3}\.){3}\d{1,3}/\d{1,2}$'
        if not re.match(ip_pattern, state['address_value']):
            errors.append(f"Invalid IP/netmask format: {state['address_value']}")
    elif state['address_type'] == "fqdn":
        # Check FQDN format
        fqdn_pattern = r'^([a-zA-Z0-9]([a-zA-Z0-9\-]{0,61}[a-zA-Z0-9])?\.)+[a-zA-Z]{2,}$'
        if not re.match(fqdn_pattern, state['address_value']):
            errors.append(f"Invalid FQDN format: {state['address_value']}")
    
    # Validate folder
    if not state['folder']:
        errors.append("Folder is required")
    
    if errors:
        print(f"❌ Validation failed: {errors}")
        return {
            "validated": False,
            "current_step": "validation_failed",
            "errors": errors
        }
    
    print(f"✅ Validation passed for {state['name']}")
    
    # Return only the fields this node updates
    return {
        "validated": True,
        "current_step": "validated",
        "errors": []
    }

# Test the node function directly
print("Testing the validate_address_input node:\n")
test_state: SCMAddressState = {
    "name": "web-server-pool",
    "address_type": "ip_netmask",
    "address_value": "10.1.1.0/24",
    "folder": "Texas",
    "description": "Web server subnet",
    "tags": ["Production"],
    "validated": False,
    "created": False,
    "object_id": None,
    "current_step": "starting",
    "errors": [],
    "api_response": None
}

result = validate_address_input(test_state)
print(f"\nNode returned updates:")
pprint(result)

print("\n💡 Notice: The node only returns the fields it cares about!")
print("   It doesn't return the entire state, just the updates.")

### 3.2 Creating Multiple Nodes for Different Tasks

In [ ]:
def prepare_scm_config(state: SCMAddressState) -> dict:
    """
    Node: Prepare configuration for SCM API.
    
    Formats the address object data according to SCM API requirements.
    
    References:
        - docs/examples/address_objects.py lines 9-15 (IP/netmask format)
        - docs/examples/address_objects.py lines 21-26 (FQDN format)
        - docs/examples/address_objects.py lines 32-37 (IP range format)
    
    Args:
        state: Current workflow state
        
    Returns:
        Dictionary with prepared API configuration
    """
    print(f"📋 Preparing SCM configuration for {state['name']}...")
    
    # Build config based on address type
    # This mirrors the config structure from docs/examples/address_objects.py
    if state['address_type'] == "ip_netmask":
        config = {
            "name": state['name'],
            "ip_netmask": state['address_value'],
            "folder": state['folder'],
            "description": state.get('description', ''),
            "tag": state.get('tags', [])
        }
    elif state['address_type'] == "fqdn":
        config = {
            "name": state['name'],
            "fqdn": state['address_value'],
            "folder": state['folder'],
            "description": state.get('description', ''),
            "tag": state.get('tags', [])
        }
    elif state['address_type'] == "ip_range":
        config = {
            "name": state['name'],
            "ip_range": state['address_value'],
            "folder": state['folder'],
            "description": state.get('description', ''),
            "tag": state.get('tags', [])
        }
    
    print(f"✅ Configuration prepared")
    
    return {
        "api_response": config,
        "current_step": "config_prepared"
    }


def create_in_scm(state: SCMAddressState) -> dict:
    """
    Node: Create address object in SCM.
    
    Simulates making an API call to create the address object.
    
    Production Usage:
        See docs/examples/address_objects.py line 18:
        netmask_address = client.address.create(netmask_config)
        
        In production, this would be:
        created_object = client.address.create(data=state['api_response'])
    
    Args:
        state: Current workflow state
        
    Returns:
        Dictionary with creation status updates
    """
    print(f"🚀 Creating address object '{state['name']}' in SCM...")
    
    # Simulate API call
    # Production: created_object = client.address.create(data=state['api_response'])
    # See docs/examples/address_objects.py lines 17-18, 28-29, 39-40
    
    import uuid
    object_id = str(uuid.uuid4())
    
    print(f"✅ Address object created successfully")
    print(f"   Object ID: {object_id}")
    
    return {
        "created": True,
        "object_id": object_id,
        "current_step": "created"
    }


def verify_creation(state: SCMAddressState) -> dict:
    """
    Node: Verify address object was created successfully.
    
    Confirms the object exists in SCM and retrieves its details.
    
    Production Usage:
        See docs/examples/address_objects.py lines 43-45:
        address = client.address.fetch(name="internal_network", folder="Texas")
        
        Or by ID (lines 47-49):
        address_by_id = client.address.get(address.id)
    
    Args:
        state: Current workflow state
        
    Returns:
        Dictionary with verification results
    """
    print(f"🔍 Verifying creation of {state['name']}...")
    
    # Simulate verification
    # Production: verified_object = client.address.get(state['object_id'])
    # See docs/examples/address_objects.py lines 47-49
    
    if state.get('object_id'):
        print(f"✅ Verified: Object exists with ID {state['object_id']}")
        verification_passed = True
    else:
        print(f"❌ Verification failed: No object ID found")
        verification_passed = False
    
    return {
        "current_step": "verified" if verification_passed else "verification_failed"
    }

print("✅ Created three SCM workflow nodes:")
print("   - prepare_scm_config: Formats data for SCM API (docs/examples/address_objects.py)")
print("   - create_in_scm: Creates address object (simulates client.address.create)")
print("   - verify_creation: Confirms object was created (simulates client.address.get)")
print("\n💡 Each node has a single, clear responsibility!")
print("   All reference the production patterns from docs/examples/address_objects.py")

### 3.2b Production Pattern: Exception Handling with SCM

In production, nodes must handle exceptions from the SCM API. The pan-scm-sdk provides specific exceptions for different error conditions that should be caught and handled gracefully.

**Common SCM Exceptions:**
- `InvalidObjectError`: Configuration data doesn't match schema
- `NameNotUniqueError`: Object name already exists in folder
- `ObjectNotPresentError`: Requested object doesn't exist
- `MissingQueryParameterError`: Required API parameter missing

Let's see how to add proper exception handling to our create node:

In [ ]:
def create_in_scm_with_exception_handling(state: SCMAddressState) -> dict:
    """
    Node: Create address object in SCM with production exception handling.
    
    Demonstrates proper error handling for SCM API operations.
    
    Production Pattern:
        See docs/examples/address_objects.py lines 170-198:
        - Import SCM exceptions (lines 158-163)
        - Try-catch block around API calls (lines 170-198)
        - Handle specific exception types
        - Return error state instead of raising
    
    Args:
        state: Current workflow state
        
    Returns:
        Dictionary with creation status and error details if failed
    """
    print(f"🚀 Creating address object '{state['name']}' in SCM (with error handling)...")
    
    # In production, import these at the top of your file
    # from scm.exceptions import InvalidObjectError, NameNotUniqueError, ObjectNotPresentError
    
    try:
        # Production code would be:
        # from scm.client import ScmClient
        # client = ScmClient(client_id="...", client_secret="...", tsg_id="...")
        # created_object = client.address.create(data=state['api_response'])
        
        # For this example, we'll simulate the API call
        import uuid
        
        # Simulate potential error scenarios for demonstration
        # In production, these would be actual API errors
        
        # Simulate checking if name already exists
        if state['name'] == "duplicate-name":
            # Simulates NameNotUniqueError from SCM
            raise Exception("NameNotUnique: Address object 'duplicate-name' already exists in folder 'Texas'")
        
        # Simulate invalid configuration
        if not state.get('api_response') or not state['api_response'].get('name'):
            # Simulates InvalidObjectError from SCM
            raise Exception("InvalidObject: Missing required field 'name' in address object configuration")
        
        # Success case - create object
        object_id = str(uuid.uuid4())
        
        print(f"✅ Address object created successfully")
        print(f"   Object ID: {object_id}")
        
        return {
            "created": True,
            "object_id": object_id,
            "current_step": "created",
            "errors": []
        }
        
    except Exception as e:
        error_msg = str(e)
        print(f"❌ Failed to create address object: {error_msg}")
        
        # Handle different exception types
        if "NameNotUnique" in error_msg:
            return {
                "created": False,
                "current_step": "create_failed",
                "errors": [f"Address object '{state['name']}' already exists in folder '{state['folder']}'"]
            }
        elif "InvalidObject" in error_msg:
            return {
                "created": False,
                "current_step": "create_failed",
                "errors": [f"Invalid configuration: {error_msg}"]
            }
        elif "ObjectNotPresent" in error_msg:
            return {
                "created": False,
                "current_step": "create_failed",
                "errors": [f"Referenced object not found: {error_msg}"]
            }
        else:
            # Catch-all for unexpected errors
            return {
                "created": False,
                "current_step": "create_failed",
                "errors": [f"Unexpected error creating address object: {error_msg}"]
            }

print("✅ Created production node with exception handling:")
print("   - Catches SCM API exceptions")
print("   - Returns error state instead of raising")
print("   - Preserves errors in state for downstream nodes")
print("   - Differentiates between error types")
print("\n💡 Production Pattern: Always use try-catch in API nodes!")
print("   See docs/examples/address_objects.py lines 170-198 for full example")

### 3.3 Testing Nodes Independently

One of the great benefits of nodes is that they're just Python functions - you can test them independently before connecting them in a graph:

In [ ]:
# Let's simulate running nodes in sequence manually
print("Simulating SCM address creation workflow by calling nodes in sequence:\n")

# Start with initial state
current_state: SCMAddressState = {
    "name": "db-server-subnet",
    "address_type": "ip_netmask",
    "address_value": "10.2.5.0/24",
    "folder": "Texas",
    "description": "Database server subnet",
    "tags": ["Production", "Database"],
    "validated": False,
    "created": False,
    "object_id": None,
    "current_step": "starting",
    "errors": [],
    "api_response": None
}

print("=" * 60)
print("STEP 1: Validate Address Input")
print("=" * 60)
validation_result = validate_address_input(current_state)
current_state = {**current_state, **validation_result}  # Merge updates

print("\n" + "=" * 60)
print("STEP 2: Prepare SCM Configuration")
print("=" * 60)
prep_result = prepare_scm_config(current_state)
current_state = {**current_state, **prep_result}

print("\n" + "=" * 60)
print("STEP 3: Create in SCM")
print("=" * 60)
create_result = create_in_scm(current_state)
current_state = {**current_state, **create_result}

print("\n" + "=" * 60)
print("STEP 4: Verify Creation")
print("=" * 60)
verify_result = verify_creation(current_state)
current_state = {**current_state, **verify_result}

print("\n" + "=" * 60)
print("FINAL STATE")
print("=" * 60)
pprint(current_state)

print("\n💡 This manual approach works, but it's tedious!")
print("   - We have to manually call each node")
print("   - We have to manually merge state updates")
print("   - The workflow order is hardcoded")
print("\n   That's where Graphs come in (next section)!")

---

## 4. Graph and Edges - Connecting the Flow

### What is StateGraph?

Before we discuss edges, let's understand **StateGraph** itself - this is one of the first and most important elements you'll interact with when building LangGraph applications.

**StateGraph** is the builder and compiler for your graph structure. Its main purpose is to:
- Build and compile the graph structure
- Manage nodes, edges, and overall state
- Ensure the workflow operates in a unified way
- Make sure data flows correctly between components

Think of StateGraph like a **configuration management playbook template**. Just as a well-designed playbook template outlines the procedure, defines all the steps, and shows how they connect together, StateGraph defines the structure and flow of your workflow. Before you can execute configuration changes in SCM, you need the playbook. Before you can run a workflow, you need StateGraph.

In practical terms:
- **Playbook template** = StateGraph (defines structure)
- **Individual tasks** = Nodes (steps in the workflow)
- **Task dependencies** = Edges (connections between steps)
- **Executing the playbook** = Running the compiled graph

StateGraph is what ties everything together - without it, you'd just have disconnected nodes and no way to orchestrate them.

### What are Edges?

Now that we understand StateGraph, let's talk about how nodes connect together. That's where **edges** come in.

**Edges** are connections between nodes that determine the flow of execution. They tell your application which node should be executed next after the current one completes its task.

Think of edges like **API call sequences in configuration workflows**. Imagine your SCM configuration workflow as a series of API calls that must happen in order - you can't create an address object before validating the input, and you can't verify creation before making the create call. Each step depends on the previous one completing successfully.

In LangGraph:
- The **API operations** = Nodes (validate, prepare, create, verify)
- The **call sequence** = Edges (connections defining order)
- The **configuration data** = State (flowing through the workflow)

Edges ensure your state flows from one processing step to the next in the correct order.

### Key Points About Edges

- **Connections**: Link nodes together to define workflow sequence
- **Directional**: Flow goes from one specific node to another
- **Determine Flow**: Control which node executes next
- **Two Types**: Simple edges (always follow the same path) and conditional edges (dynamic routing)

### Why Edges Matter

Without edges:
- Nodes would be disconnected and isolated
- You'd have to manually orchestrate the execution order
- No way to define the workflow sequence

With edges:
- Workflow sequence is explicit and visual
- State automatically flows from node to node
- Easy to modify the workflow by changing edges

### 4.1 Building Your First Graph with StateGraph

Now let's put it all together! We'll use **StateGraph** to create a graph, add our nodes, and connect them with edges.

In [ ]:
# Step 1: Create a StateGraph
# This is the container for our SCM workflow
scm_graph = StateGraph(SCMAddressState)

# Step 2: Add nodes to the graph
# Each node is given a name and associated with a function
scm_graph.add_node("validate", validate_address_input)
scm_graph.add_node("prepare", prepare_scm_config)
scm_graph.add_node("create", create_in_scm)
scm_graph.add_node("verify", verify_creation)

print("✅ Graph created with 4 nodes:")
print("   - validate: Validates address object input")
print("   - prepare: Prepares SCM API configuration")
print("   - create: Creates object in SCM (simulated)")
print("   - verify: Verifies object was created")
print("\n💡 But the nodes aren't connected yet - we need edges!")

### 4.2 Understanding START and END

Before we connect nodes with edges, let's understand two special elements: **START** and **END**.

**START** is a virtual entry point in LangGraph that marks where the workflow begins. It's important to note that START doesn't perform any operations itself - it simply serves as the designated starting position for the graph's execution. Think of it like the **beginning of your configuration playbook** - it's where you open the document and start following the procedure.

**END** signifies the conclusion of the workflow in LangGraph. When the application reaches this point, the graph's execution completely stops, indicating that all intended processes have been completed. This is like reaching the **final step in your playbook** - the configuration change is complete and there's nothing more to do.

In SCM automation terms:
- **START** = The moment you begin executing your configuration workflow
- **END** = The completion of all configuration tasks and validations

### 4.3 Adding Edges to Define Workflow Flow

Now we'll connect the nodes with edges, using START as our entry point and END as our exit point:

In [ ]:
# Step 3: Add edges to connect the nodes
# Edges define the flow: START → validate → prepare → create → verify → END

# Entry point: Start the workflow at validate
scm_graph.add_edge(START, "validate")

# Connect the nodes in sequence
scm_graph.add_edge("validate", "prepare")
scm_graph.add_edge("prepare", "create")
scm_graph.add_edge("create", "verify")

# Exit point: End the workflow after verify
scm_graph.add_edge("verify", END)

print("✅ Edges added to create SCM workflow sequence")
print("\nVisualizing the graph structure:")

### 4.3b Step-by-Step Graph Construction

Before we compile and run the graph, let's see exactly what happens during graph construction. This intermediate step helps visualize how StateGraph builds your workflow structure piece by piece.

In [ ]:
# Let's rebuild the graph step-by-step to see what's happening

print("Step 1: Create empty StateGraph")
print("=" * 60)
step_graph = StateGraph(SCMAddressState)
print("✅ Empty graph created for SCMAddressState")
print("   Graph has no nodes or edges yet - it's just a container\n")

print("Step 2: Add first node (validate)")
print("=" * 60)
step_graph.add_node("validate", validate_address_input)
print("✅ Added 'validate' node")
print("   Function: validate_address_input")
print("   Node is registered but not connected to anything yet\n")

print("Step 3: Add remaining nodes")
print("=" * 60)
step_graph.add_node("prepare", prepare_scm_config)
print("✅ Added 'prepare' node")

step_graph.add_node("create", create_in_scm)
print("✅ Added 'create' node")

step_graph.add_node("verify", verify_creation)
print("✅ Added 'verify' node")
print("   All 4 nodes registered, but still disconnected\n")

print("Step 4: Connect START to first node")
print("=" * 60)
step_graph.add_edge(START, "validate")
print("✅ Connected START → validate")
print("   Workflow will begin at 'validate' node\n")

print("Step 5: Connect nodes in sequence")
print("=" * 60)
step_graph.add_edge("validate", "prepare")
print("✅ Connected validate → prepare")

step_graph.add_edge("prepare", "create")
print("✅ Connected prepare → create")

step_graph.add_edge("create", "verify")
print("✅ Connected create → verify")
print("   Nodes now form a pipeline\n")

print("Step 6: Connect last node to END")
print("=" * 60)
step_graph.add_edge("verify", END)
print("✅ Connected verify → END")
print("   Workflow will exit after 'verify' completes\n")

print("=" * 60)
print("Graph Structure Complete!")
print("=" * 60)
print("\n📊 Current graph state:")
print("   Nodes: validate, prepare, create, verify")
print("   Edges: START→validate→prepare→create→verify→END")
print("\n💡 Graph is defined but NOT yet executable")
print("   Next step: Compile to create a Runnable")

### 4.3 Understanding Runnables

Now that we've added edges to connect our nodes, let's talk about what happens when we compile the graph: we create a **Runnable**.

If you're coming from a LangChain background, you're probably familiar with Runnables - they work similarly in LangGraph.

**A Runnable** in LangGraph is a standardized executable component that performs a specific task within your workflow. It acts as a fundamental building block, allowing you to create modular systems. When you compile a StateGraph, you get back a Runnable that can execute your entire workflow.

**Runnable vs Node - What's the Difference?**

You might be wondering: "What's the difference between a Runnable and a Node?"

Short answer:
- **Runnable** can represent various operations - it's a general execution interface
- **Node** in LangGraph typically receives state, performs an action, and updates state

Think of it this way in our SCM address object workflow context:
- **Node** = A specific step (validate, prepare, create, verify)
- **Runnable** = The entire workflow procedure that can be executed
- **Compiled Graph (app)** = A Runnable that orchestrates all nodes

Here's a useful analogy: Think of Runnables like **modular rack-mounted equipment**. Just as you can snap together different rack units (firewall, switch, router, storage) to build a complete data center infrastructure, Runnables can be combined to create sophisticated automation workflows. Each piece has a standard interface (rack mount) and can be swapped or combined as needed.

Don't worry if this isn't 100% clear yet - you'll understand it better as we compile and use the graph in the next section!

💡 **Key Insight about START and END:**
- START is not a real node - it's a marker for where execution begins
- END is not a real node - it's a marker for where execution stops  
- They don't process state or perform operations
- Every graph must have at least one edge from START and one edge to END

In [ ]:
# Compile the graph to create a runnable application
scm_app = scm_graph.compile()

# Display the graph structure as a mermaid diagram
display(Image(scm_app.get_graph().draw_mermaid_png()))

print("\n💡 The graph visualization shows:")
print("   - Rounded rectangles: Processing nodes")
print("   - Arrows: Edges (flow direction)")
print("   - __start__: Entry point (START)")
print("   - __end__: Exit point (END)")
print("\n   State flows from START through each node to END in sequence")
print("\n✨ This is why LangGraph workflows are visual and easy to understand!")
print("   You can see the entire configuration pipeline at a glance!")

### 4.4 Executing the Runnable

Now let's use our compiled Runnable to execute the SCM address creation workflow!

In [ ]:
# Define initial state for SCM address creation
initial_state: SCMAddressState = {
    "name": "production-web-servers",
    "address_type": "ip_netmask",
    "address_value": "10.50.100.0/24",
    "folder": "Texas",
    "description": "Production web server network",
    "tags": ["Production", "WebServers", "Automation"],
    "validated": False,
    "created": False,
    "object_id": None,
    "current_step": "starting",
    "errors": [],
    "api_response": None
}

print("Starting SCM Address Creation Workflow")
print("=" * 60)
print("\nInitial State:")
pprint(initial_state)

print("\n" + "=" * 60)
print("Executing Runnable (compiled graph)...")
print("=" * 60 + "\n")

# Invoke the Runnable - it will automatically execute all nodes in order
final_state = scm_app.invoke(initial_state)

print("\n" + "=" * 60)
print("Workflow Complete!")
print("=" * 60)
print("\nFinal State:")
pprint(final_state)

print("\n✅ The Runnable (scm_app) automatically:")
print("   - Executed each node in the correct order defined by edges")
print("   - Merged state updates from each node")
print("   - Returned the final state")
print("\n💡 Compare this to the manual approach in section 3.3!")
print("   The Runnable interface makes execution clean and simple!")
print("\n✨ Key Benefits of Graph-based Workflows:")
print("   1. Visual - You can SEE the workflow structure")
print("   2. Explicit - State flow is clearly defined")
print("   3. Modifiable - Easy to add/remove/reorder steps")

### 4.6 Practical Example: SCM Address Object Workflow

Now let's build a simpler, focused example using **SCM address objects**. This demonstrates the classic 3-node pattern: **parse → validate → format** that's common in configuration workflows.

This example shows how to transform user input into API-ready SCM configurations!

In [ ]:
# Step 1: Define State Schema for Address Object Workflow
class AddressObjectState(TypedDict):
    """
    State schema for SCM address object creation workflow.
    
    This mirrors the structure from docs/examples/address_objects.py
    """
    # Input
    raw_input: str  # User's text input
    
    # Parsed fields
    name: str
    ip_netmask: str
    folder: str
    description: str
    
    # Validation
    validated: bool
    error_message: str
    
    # Final output
    api_ready_config: dict

print("✅ AddressObjectState schema defined")
print("\nThis state tracks the transformation:")
print("  Raw input → Parsed fields → Validated → API-ready config")

In [ ]:
# Step 2: Create the three workflow nodes

def parse_input(state: AddressObjectState) -> dict:
    """
    Node 1: Parse raw user input into structured fields.
    
    Expects input format: "create address <name> <ip/mask> in <folder>"
    Example: "create address web-server 10.1.1.100/32 in Texas"
    
    Args:
        state: Current workflow state
        
    Returns:
        Dictionary with parsed fields
    """
    print(f"📝 Parsing input: {state['raw_input']}")
    
    # Simple parsing logic (in production, use regex or proper parser)
    parts = state["raw_input"].split()
    
    if len(parts) < 6:
        return {
            "validated": False,
            "error_message": "Invalid input format. Expected: create address <name> <ip/mask> in <folder>"
        }
    
    name = parts[2]
    ip_netmask = parts[3]
    folder = parts[5]
    description = f"Address object for {name}"
    
    print(f"✅ Parsed: name={name}, ip_netmask={ip_netmask}, folder={folder}")
    
    return {
        "name": name,
        "ip_netmask": ip_netmask,
        "folder": folder,
        "description": description,
        "error_message": ""
    }


def validate_address(state: AddressObjectState) -> dict:
    """
    Node 2: Validate parsed address object fields.
    
    Checks:
    - IP address format is valid
    - Required fields are present
    - Folder name is valid
    
    Args:
        state: Current workflow state
        
    Returns:
        Dictionary with validation results
    """
    import re
    
    print(f"🔍 Validating address object: {state.get('name', 'unknown')}")
    
    # Check if parsing failed
    if state.get("error_message"):
        return {"validated": False}
    
    # Validate IP/netmask format
    ip_pattern = r'^(\d{1,3}\.){3}\d{1,3}/\d{1,2}$'
    
    if not re.match(ip_pattern, state["ip_netmask"]):
        print(f"❌ Invalid IP format: {state['ip_netmask']}")
        return {
            "validated": False,
            "error_message": f"Invalid IP format: {state['ip_netmask']}"
        }
    
    # Validate required fields are present
    if not state.get("name") or not state.get("folder"):
        print("❌ Missing required fields")
        return {
            "validated": False,
            "error_message": "Missing required fields (name or folder)"
        }
    
    print(f"✅ Validation passed for {state['name']}")
    return {
        "validated": True,
        "error_message": ""
    }


def format_for_api(state: AddressObjectState) -> dict:
    """
    Node 3: Transform validated state into SCM API format.
    
    Creates configuration matching the structure from docs/examples/address_objects.py.
    
    Production Pattern:
        See docs/examples/address_objects.py lines 9-15:
        netmask_config = {
            "name": "internal_network",
            "ip_netmask": "192.168.1.0/24",
            "description": "Internal network segment",
            "folder": "Texas",
            "tag": ["Python", "Automation"],
        }
    
    Args:
        state: Current workflow state
        
    Returns:
        Dictionary with API-ready configuration
    """
    print(f"🔧 Formatting configuration for SCM API")
    
    if not state["validated"]:
        print("⚠️  Skipping format - validation failed")
        return {"api_ready_config": {}}
    
    # Format according to docs/examples/address_objects.py lines 9-15
    # This config structure matches the pan-scm-sdk address object schema
    config = {
        "name": state["name"],
        "ip_netmask": state["ip_netmask"],
        "folder": state["folder"],
        "description": state["description"],
        "tag": ["Automation", "LangGraph"]
    }
    
    print(f"✅ API-ready configuration created:")
    pprint(config)
    
    return {"api_ready_config": config}


print("✅ Created three workflow nodes:")
print("   1. parse_input: Extracts fields from user input")
print("   2. validate_address: Validates IP format and required fields")
print("   3. format_for_api: Creates SCM API-ready configuration")
print("\n💡 The format_for_api node mirrors docs/examples/address_objects.py structure!")

In [ ]:
# Step 3: Build the graph with START → parse → validate → format → END

# Create StateGraph for address object workflow
address_graph = StateGraph(AddressObjectState)

# Add the three nodes
address_graph.add_node("parse", parse_input)
address_graph.add_node("validate", validate_address)
address_graph.add_node("format", format_for_api)

# Connect nodes in sequence
address_graph.add_edge(START, "parse")
address_graph.add_edge("parse", "validate")
address_graph.add_edge("validate", "format")
address_graph.add_edge("format", END)

# Compile to create runnable
address_app = address_graph.compile()

print("✅ SCM Address Object graph created and compiled")
print("\nGraph structure:")
print("  START → parse → validate → format → END")
print("\nVisualizing graph:")

In [ ]:
# Display the graph visualization
display(Image(address_app.get_graph().draw_mermaid_png()))

print("\n💡 This is the classic parse → validate → format pattern")
print("   used throughout configuration management workflows!")

In [ ]:
# Step 4: Test with valid input

print("="*60)
print("TEST 1: Valid Address Object Input")
print("="*60 + "\n")

valid_input: AddressObjectState = {
    "raw_input": "create address web-server 10.1.1.100/32 in Texas",
    "name": "",
    "ip_netmask": "",
    "folder": "",
    "description": "",
    "validated": False,
    "error_message": "",
    "api_ready_config": {}
}

print("Input:", valid_input["raw_input"])
print()

# Execute the workflow
result = address_app.invoke(valid_input)

print("\n" + "="*60)
print("FINAL STATE")
print("="*60)
pprint(result)

print("\n✅ Success! The workflow:")
print("   1. Parsed user input into structured fields")
print("   2. Validated IP address format")
print("   3. Created API-ready SCM configuration")
print("\n   This config is ready to send to the pan-scm-sdk!")

In [ ]:
# Step 5: Test with invalid input to see error handling

print("\n" + "="*60)
print("TEST 2: Invalid IP Format")
print("="*60 + "\n")

invalid_input: AddressObjectState = {
    "raw_input": "create address db-server 10.1.1.100 in Texas",  # Missing /mask!
    "name": "",
    "ip_netmask": "",
    "folder": "",
    "description": "",
    "validated": False,
    "error_message": "",
    "api_ready_config": {}
}

print("Input:", invalid_input["raw_input"])
print("Note: IP is missing the subnet mask!\n")

# Execute the workflow
result2 = address_app.invoke(invalid_input)

print("\n" + "="*60)
print("FINAL STATE")
print("="*60)
pprint(result2)

print("\n💡 The workflow caught the error:")
print(f"   validated: {result2['validated']}")
print(f"   error_message: {result2['error_message']}")
print("   api_ready_config: {} (empty - not created)")
print("\n   Validation prevented bad data from reaching the API!")

In [ ]:
print("\n" + "="*60)
print("Key Takeaways from SCM Address Object Workflow")
print("="*60)

print("""
This simple 3-node graph demonstrates fundamental LangGraph patterns:

1. **State Schema** (AddressObjectState)
   - Tracks the transformation from raw input to API-ready config
   - Uses TypedDict from Notebook 101 for type safety

2. **Sequential Nodes** (parse → validate → format)
   - Each node has a single responsibility
   - Node output updates state for next node
   - Classic configuration pipeline pattern

3. **Validation Gates**
   - Validation node checks data before proceeding
   - Invalid data stops the pipeline (validated=False)
   - Prevents bad configs from reaching the API

4. **API Integration Pattern**
   - Final node creates config matching pan-scm-sdk structure
   - From docs/examples/address_objects.py
   - Ready to send to SCM API

5. **Real-World Application**
   - User inputs text command
   - Graph transforms it into validated API request
   - Production-ready automation pattern

This is the foundation you'll use for more complex workflows:
- Security rule creation and validation
- Configuration change workflows
- Multi-step automation with conditional routing
- Error handling and rollback procedures

💡 In the next sections, you'll see how to add conditional routing
   and tool integration to make workflows even more powerful!
""")

### 4.7 Production Pattern: Commit Workflow

In production SCM workflows, creating an address object is only half the story. Changes must be **committed** to push them from the candidate configuration to the running configuration.

**SCM Commit Workflow:**
1. Create/update objects (candidate configuration)
2. Commit changes to folders
3. Monitor commit job status
4. Verify job completed successfully

Let's add a commit node to our workflow to demonstrate the complete end-to-end pattern:

In [ ]:
# First, extend our state to track commit information
class SCMAddressStateWithCommit(TypedDict):
    """Extended state schema including commit tracking."""
    # Input data
    name: str
    address_type: str
    address_value: str
    folder: str
    description: Optional[str]
    tags: list[str]
    
    # Workflow tracking
    validated: bool
    created: bool
    object_id: Optional[str]
    current_step: str
    errors: list[str]
    
    # API response data
    api_response: Optional[dict]
    
    # Commit tracking (new fields)
    committed: bool
    commit_job_id: Optional[str]
    commit_status: str


def commit_changes(state: SCMAddressStateWithCommit) -> dict:
    """
    Node: Commit changes to SCM.
    
    Pushes candidate configuration to running configuration.
    
    Production Pattern:
        See docs/examples/address_objects.py lines 183-186:
        result = client.commit(
            folders=["Texas"],
            description="Added test address",
            sync=True
        )
        
        Then check job status (lines 188-189):
        status = client.get_job_status(result.job_id)
    
    Args:
        state: Current workflow state
        
    Returns:
        Dictionary with commit status updates
    """
    print(f"📤 Committing changes to folder '{state['folder']}'...")
    
    if not state.get('created'):
        print("⚠️  Skipping commit - object not created")
        return {
            "committed": False,
            "commit_status": "skipped",
            "current_step": "commit_skipped"
        }
    
    try:
        # Production code:
        # from scm.client import ScmClient
        # client = ScmClient(...)
        # result = client.commit(
        #     folders=[state['folder']],
        #     description=f"Created address object {state['name']}",
        #     sync=True  # Wait for job completion
        # )
        # job_status = client.get_job_status(result.job_id)
        
        # Simulate commit
        import uuid
        import time
        
        commit_job_id = str(uuid.uuid4())
        print(f"   Job ID: {commit_job_id}")
        print(f"   Committing to folder: {state['folder']}")
        
        # Simulate job processing
        time.sleep(0.5)
        
        # Simulate checking job status
        job_status = "FIN"  # "FIN" = finished successfully
        
        if job_status == "FIN":
            print(f"✅ Commit completed successfully")
            return {
                "committed": True,
                "commit_job_id": commit_job_id,
                "commit_status": "completed",
                "current_step": "committed"
            }
        else:
            print(f"❌ Commit failed with status: {job_status}")
            return {
                "committed": False,
                "commit_job_id": commit_job_id,
                "commit_status": f"failed: {job_status}",
                "current_step": "commit_failed",
                "errors": state.get('errors', []) + [f"Commit job failed: {job_status}"]
            }
            
    except Exception as e:
        print(f"❌ Commit error: {str(e)}")
        return {
            "committed": False,
            "commit_status": "error",
            "current_step": "commit_failed",
            "errors": state.get('errors', []) + [f"Commit error: {str(e)}"]
        }

print("✅ Created commit_changes node")
print("\n💡 Complete SCM Workflow:")
print("   1. Validate address object data")
print("   2. Prepare SCM API configuration")
print("   3. Create object (candidate config)")
print("   4. Commit changes (push to running config)")
print("   5. Verify commit job completed")
print("\n📚 Production Reference: docs/examples/address_objects.py lines 183-189")

---

## 5. What's Next: Conditional Routing

So far, we've built workflows with **simple sequential edges** - our graphs follow a fixed path from START to END. But what if you need your workflow to make decisions and take different paths based on the data?

That's where **conditional routing** comes in! In **Notebook 103**, you'll learn how to:

- Add **conditional edges** that route based on state values
- Create **router functions** that decide which path to take (like if/else logic)
- Handle **success and error paths** differently (e.g., retry on failure, skip steps if not needed)
- Build **intelligent workflows** that adapt based on validation results

### Example Preview: Validation-Based Routing

Imagine you want your workflow to handle validation failures differently than successes:

**Current Simple Edge Approach:**
```
START → validate → prepare → create → END
```

Problem: If validation fails, we still try to prepare and create!

**With Conditional Routing:**
```
                    ┌─ validation success ─> prepare → create → END
START → validate ───┤
                    └─ validation failure ──> error_handler → END
```

**How It Works:**

1. **Router Function** - Decides which path to take:
```python
def route_after_validation(state: SCMAddressState) -> str:
    \"\"\"Route based on validation results.
    
    Returns:
        str: Name of the next node to execute
    \"\"\"
    if state["validated"]:
        return "prepare"  # Success path
    else:
        return "error_handler"  # Error path
```

2. **Add Conditional Edge** - Use router to choose next node:
```python
from langgraph.graph import StateGraph, START, END

# Build graph
graph = StateGraph(SCMAddressState)
graph.add_node("validate", validate_address_input)
graph.add_node("prepare", prepare_scm_config)
graph.add_node("error_handler", handle_validation_error)

# Simple edges
graph.add_edge(START, "validate")

# CONDITIONAL edge - chooses next node dynamically
graph.add_conditional_edges(
    "validate",  # After this node completes
    route_after_validation,  # Call this router function
    {
        "prepare": "prepare",  # If returns "prepare", go to prepare node
        "error_handler": "error_handler"  # If returns "error_handler", go there
    }
)

# Continue with success path
graph.add_edge("prepare", "create")
graph.add_edge("create", END)

# Error path ends
graph.add_edge("error_handler", END)
```

### Real-World SCM Scenarios for Conditional Routing

**Scenario 1: IP Address Type Detection**
```python
def route_by_address_type(state: SCMAddressState) -> str:
    \"\"\"Route to different validation logic based on address type.\"\"\"
    if state["address_type"] == "ip_netmask":
        return "validate_ip"
    elif state["address_type"] == "fqdn":
        return "validate_fqdn"
    elif state["address_type"] == "ip_range":
        return "validate_range"
    else:
        return "error"
```

Graph flow:
```
START → parse_input → [route by type] ─┬─ ip_netmask ──> validate_ip ──┐
                                        ├─ fqdn ───────> validate_fqdn ─┤
                                        ├─ ip_range ───> validate_range ┤
                                        └─ error ──────> error_handler ─┘
                                                                         ↓
                                                                    format → END
```

**Scenario 2: Retry Logic on API Failure**
```python
def route_after_create(state: SCMAddressState) -> str:
    \"\"\"Retry creation if it failed, up to 3 times.\"\"\"
    if state["created"]:
        return "verify"  # Success - move to verification
    elif state.get("retry_count", 0) < 3:
        return "retry_create"  # Retry
    else:
        return "failure_handler"  # Give up after 3 retries
```

**Scenario 3: Security Policy Rule Complexity**
```python
def route_by_rule_complexity(state: SecurityPolicyState) -> str:
    \"\"\"Route complex rules to enhanced validation.\"\"\"
    has_profiles = bool(state.get("profile_setting"))
    has_schedule = bool(state.get("schedule"))
    many_apps = len(state.get("application", [])) > 10
    
    if has_profiles or has_schedule or many_apps:
        return "enhanced_validation"  # Complex rule needs extra checks
    else:
        return "basic_validation"  # Simple rule can use fast path
```

### Why Conditional Routing Matters

Think of conditional routing like **network ACL logic** - matching conditions and taking different actions based on the match result.

**Without Conditional Routing:**
- All configurations follow the same path (even when validation fails)
- Can't handle different address types efficiently
- No retry logic for transient failures
- Error handling happens too late

**With Conditional Routing:**
- Intelligent branching based on data
- Different validation logic for different types
- Automatic retry for temporary failures
- Early error detection and handling

### Key Components

1. **Router Function**: Pure function that examines state and returns next node name
2. **Conditional Edge**: `add_conditional_edges(source, router, mapping)`
3. **Path Mapping**: Dictionary mapping router return values to node names

**Coming in Notebook 103:** We'll integrate this with **LLM-powered decision making** to build a conversational chatbot that can:
- Route user requests to appropriate handlers
- Decide when to use tools vs when to respond directly
- Handle multi-turn conversations with context
- Make intelligent decisions based on natural language understanding

For now, let's continue with our foundational concepts...

---

## 6. What's Next: Tools and External Capabilities

So far, we've used simple Python functions as nodes. But what if you need your workflow to interact with external systems like:
- Making API calls to Strata Cloud Manager
- Validating IP addresses with advanced logic
- Checking if folders exist in SCM
- Creating, reading, updating, or deleting configuration objects

That's where **Tools** come in! In **Notebook 106**, you'll learn how to:

- Create **reusable tools** with the `@tool` decorator
- Integrate **pan-scm-sdk methods** as tools (create, list, fetch, update, delete)
- Use **ToolNode** to execute tools within graphs
- Build **ReAct agents** that intelligently select and use tools
- Enable **LLMs to call tools** based on natural language requests

### Understanding the Distinction: Nodes vs Tools

**Key Difference:**

**Nodes** are workflow orchestration functions:
- Control the flow of your graph
- Update state based on current values
- Make decisions about what happens next
- Part of your graph structure (added with `add_node()`)
- Example: A validation node that checks data and routes to success or error paths

**Tools** are reusable operations:
- Perform specific external operations (API calls, database queries, calculations)
- Can be called dynamically by LLM agents
- Not part of the graph structure (passed to agents or ToolNode)
- Decorated with `@tool` to enable LLM usage
- Example: A tool that creates an address object via `client.address.create()`

**Real-World Analogy:**
Think of nodes like **managers** who coordinate work, and tools like **specialists** who do specific technical tasks. The manager (node) decides *when* to use a specialist, but the specialist (tool) has the expertise to actually perform the technical operation.

### Example Preview: pan-scm-sdk Methods as Tools

Here are examples of how pan-scm-sdk operations would be wrapped as tools:

```python
from langchain_core.tools import tool
from scm.client import ScmClient

@tool
def create_address_object(name: str, ip_netmask: str, folder: str, description: str = "") -> dict:
    \"\"\"Create an address object in Strata Cloud Manager.
    
    Args:
        name: Address object name
        ip_netmask: IP address with CIDR notation (e.g., '10.1.1.0/24')
        folder: SCM folder name (e.g., 'Texas')
        description: Optional description
        
    Returns:
        Dictionary with created object details including ID
    \"\"\"
    client = ScmClient(client_id="...", client_secret="...", tsg_id="...")
    
    config = {
        "name": name,
        "ip_netmask": ip_netmask,
        "folder": folder,
        "description": description
    }
    
    # See docs/examples/address_objects.py line 18
    created_object = client.address.create(data=config)
    
    return {
        "id": created_object.id,
        "name": created_object.name,
        "folder": created_object.folder
    }


@tool
def list_address_objects(folder: str) -> list:
    \"\"\"List all address objects in a specific folder.
    
    Args:
        folder: SCM folder name to query
        
    Returns:
        List of address object dictionaries
    \"\"\"
    client = ScmClient(client_id="...", client_secret="...", tsg_id="...")
    
    # See docs/examples/address_objects.py lines 53-54
    addresses = client.address.list(folder=folder)
    
    return [
        {"name": addr.name, "ip_netmask": addr.ip_netmask, "folder": addr.folder}
        for addr in addresses
    ]


@tool
def fetch_address_object(name: str, folder: str) -> dict:
    \"\"\"Retrieve a specific address object by name and folder.
    
    Args:
        name: Address object name
        folder: SCM folder name
        
    Returns:
        Dictionary with address object details
    \"\"\"
    client = ScmClient(client_id="...", client_secret="...", tsg_id="...")
    
    # See docs/examples/address_objects.py lines 43-45
    address = client.address.fetch(name=name, folder=folder)
    
    return {
        "id": address.id,
        "name": address.name,
        "ip_netmask": address.ip_netmask,
        "folder": address.folder,
        "description": address.description
    }


@tool
def update_address_object(object_id: str, description: str) -> dict:
    \"\"\"Update an address object's description.
    
    Args:
        object_id: UUID of the address object
        description: New description text
        
    Returns:
        Dictionary with updated object details
    \"\"\"
    client = ScmClient(client_id="...", client_secret="...", tsg_id="...")
    
    # See docs/examples/address_objects.py lines 60-67
    existing_address = client.address.get(object_id)
    existing_address.description = description
    updated_address = client.address.update(existing_address)
    
    return {
        "id": updated_address.id,
        "name": updated_address.name,
        "description": updated_address.description
    }


@tool
def delete_address_object(object_id: str) -> dict:
    \"\"\"Delete an address object from SCM.
    
    Args:
        object_id: UUID of the address object to delete
        
    Returns:
        Dictionary with deletion status
    \"\"\"
    client = ScmClient(client_id="...", client_secret="...", tsg_id="...")
    
    # See docs/examples/address_objects.py line 75
    client.address.delete(object_id)
    
    return {"status": "deleted", "id": object_id}
```

### How Tools Integrate with Graphs

In **Notebook 106**, you'll learn to use these tools in two ways:

**1. ToolNode in Graphs:**
```python
from langgraph.prebuilt import ToolNode

# Create a list of tools
scm_tools = [
    create_address_object,
    list_address_objects,
    fetch_address_object
]

# Add ToolNode to graph
graph.add_node("tools", ToolNode(scm_tools))

# Connect to tools
graph.add_edge("agent", "tools")
graph.add_edge("tools", "agent")
```

**2. ReAct Agents with Tools:**
```python
from langgraph.prebuilt import create_react_agent

# Agent can intelligently select and use tools
agent = create_react_agent(
    model=llm,
    tools=scm_tools,
    state_schema=SCMAddressState
)

# Agent decides which tool to use based on user request
result = agent.invoke({
    "messages": ["Create an address object named web-srv with IP 10.1.1.100/32 in Texas folder"]
})
```

**Coming in Notebook 106:** You'll build autonomous agents that can:
- Understand natural language requests
- Select the appropriate SCM tool automatically
- Execute complex multi-step operations
- Handle errors and retry with different approaches

For now, let's see how these core concepts come together...

---

## 7. Messages in LangGraph

> **📘 Preview Note**: This section introduces Messages as a preview for **Notebook 103** where you'll integrate LLMs into your workflows. Messages aren't required for the basic automation workflows in this notebook, but understanding them now will prepare you for building AI-powered agents in the next notebooks!

### What are Messages?

When building LangGraph applications that interact with Large Language Models (LLMs), you'll work with different types of **messages**. If you're coming from a LangChain background, you'll be quite familiar with these. If not, don't worry - we'll cover the five most common message types in LangGraph.

Messages are how you communicate with LLMs in a structured way. Think of them like **different types of firewall logs** - each has a specific format and purpose:
- **Traffic logs** = Different information than threat logs
- **System logs** = Different from configuration logs
- Each log type has its specific structure and use case

Similarly, each message type in LangGraph serves a specific purpose in your AI workflow.

### The Five Common Message Types

1. **HumanMessage** - Represents input from a user
   - Like a **ticket from a network engineer**: "Check the HA status of fw01"
   - This is what the user wants the AI to do

2. **AIMessage** - Represents responses generated by AI models  
   - Like the **AI's response**: "I've checked fw01. HA is active and synced."
   - This is what the AI says back

3. **SystemMessage** - Used to provide instructions or context to the model
   - Like **standard operating procedures**: "You are a PAN-OS firewall expert. Always check HA before upgrades."
   - Sets the behavior and context for the AI

4. **ToolMessage** - Specific to tool usage, contains results from tool execution
   - Like **command output**: The result from running `show high-availability state`
   - Contains actual data returned by tools

5. **FunctionMessage** - Represents the result of a function call (similar to ToolMessage)
   - Like **API response data**: The JSON returned from a PAN-OS API call
   - Contains structured data from function executions

### When You'll Use Messages

If you've used LLM APIs before (like OpenAI's API or Anthropic's Claude API), you'll recognize HumanMessage, AIMessage, and SystemMessage - they're fundamental to conversational AI.

In our SCM address object workflow examples, we didn't use messages because we weren't interacting with an LLM. But in a more advanced workflow where an AI agent makes decisions about configurations, you'd use:
- **SystemMessage**: "You are an SCM configuration specialist..."
- **HumanMessage**: "Create an address object for web-server 10.1.1.100/32 in Texas folder"
- **AIMessage**: "I'll parse and validate the input first..."
- **ToolMessage**: Results from validating IP format, checking folder existence, etc.

### Key Takeaway

Messages are essential when building **conversational** or **agent-based** workflows with LLMs. For now, just know they exist and understand their purposes. You'll use them extensively when we add LLM agents to our workflows in **Notebook 103**!

💡 **Coming in Notebook 103**: You'll see Messages in action when we build an LLM-powered agent that can:
- Understand natural language requests for SCM configurations
- Use tools dynamically based on the request
- Provide conversational responses about configuration status
- Make intelligent decisions about which workflow path to follow

---

## 8. Hands-On Exercises

Now that you've learned the core concepts, it's time to put them into practice! These exercises will reinforce your understanding and prepare you for more advanced notebooks.

### Exercise 1: Build a 3-Node Address Object Pipeline

**Objective**: Create a complete parse → validate → format workflow for address objects

**Requirements**:
1. Define an `AddressObjectState` TypedDict with fields:
   - `raw_input`: str (user's text input)
   - `name`: str (parsed object name)
   - `ip_netmask`: str (parsed IP/netmask)
   - `folder`: str (parsed folder name)
   - `validated`: bool (validation status)
   - `error_message`: str (error details if validation fails)
   - `api_ready_config`: dict (final SCM API configuration)

2. Create three nodes:
   - `parse_input`: Extract name, IP/netmask, folder from raw input
   - `validate_address`: Validate IP format and required fields
   - `format_for_api`: Create API-ready configuration matching docs/examples/address_objects.py

3. Build a StateGraph:
   - Add all three nodes
   - Connect with simple edges: START → parse → validate → format → END
   - Compile and test with valid and invalid inputs

**Success Criteria**:
- Valid input produces complete API-ready configuration
- Invalid input is caught by validation and reported in error_message
- State flows correctly through all nodes
- Graph visualization shows clean pipeline structure

---

### Exercise 2: Security Policy State Schema

**Objective**: Design a TypedDict for security policy creation workflow

**Requirements**:
1. Create a `SecurityPolicyState` TypedDict with fields for:
   - **Input**: name, description
   - **Zones**: from_zones (list), to_zones (list)
   - **Addresses**: source (list), destination (list)
   - **Application**: application (list)
   - **Action**: action (str - "allow", "deny", or "drop")
   - **Tracking**: validated (bool), created (bool), errors (list)
   - **Output**: api_ready_config (dict)

2. Create a validation node that checks:
   - Name is provided and non-empty
   - At least one from_zone and one to_zone
   - At least one source and one destination
   - Action is one of: "allow", "deny", "drop"
   - Returns validation status and any errors

3. Test the validation node with various inputs:
   - Complete valid policy
   - Missing name
   - Invalid action
   - Missing zones

**Success Criteria**:
- TypedDict includes all necessary fields
- Validation node catches all error conditions
- Error messages are clear and specific
- State structure supports full policy creation workflow

---

### Exercise 3: Add Error Handling Node

**Objective**: Extend Exercise 1 with error detection and handling

**Requirements**:
1. Take your Exercise 1 solution
2. Add an `error_handler` node that:
   - Receives state with validation errors
   - Formats error message for user
   - Logs error details
   - Sets current_step to "failed"

3. Modify the workflow:
   - Keep: START → parse → validate → format → END (success path)
   - The format node should check if `validated` is True
   - If validated is False, format node should set final state indicating failure
   - Error information should be preserved in state

**Success Criteria**:
- Successful validation flows to format node
- Failed validation is handled gracefully
- Error messages are preserved and clear
- State shows final status (success vs failed)
- Graph remains easy to understand

---

### Bonus Challenge: Progressive Complexity Levels

Build the same address object workflow at four complexity levels:

**Level 1 - Single Node:**
- One node that does everything: parse, validate, and format
- Minimal state (just input and output)
- No error handling

**Level 2 - Sequential (2-3 nodes):**
- Parse → Validate → Format pipeline
- State accumulates through nodes
- Each node has single responsibility

**Level 3 - State Transformations:**
- Add complex field validations (IP octet ranges, CIDR limits)
- Data type conversions (string lists to arrays)
- Conditional updates based on address_type

**Level 4 - Visualization:**
- Implement all of Level 3
- Add graph visualization with draw_mermaid_png()
- Add state logging at each node
- Add comprehensive error messages

**Success Criteria**:
- Each level works correctly
- Progression shows increasing complexity
- Level 4 is production-ready quality
- You understand trade-offs between levels

---

### Tips for Success

1. **Start Simple**: Get Level 1 working before moving to Level 2
2. **Test Frequently**: Run your nodes independently before connecting in graph
3. **Visualize**: Always use `draw_mermaid_png()` to see your graph structure
4. **Reference Examples**: Look at the complete examples in this notebook
5. **Check State**: Print state at each node to verify data flow
6. **Error First**: Test error cases before success cases

### Solutions

Solutions to these exercises are available in the `solutions/` directory. Try to complete them on your own first - the learning happens in the struggle!

**Next Steps**: Once you've completed these exercises, you're ready for **Notebook 103** where you'll add conditional routing and LLM integration!

---

## 9. Summary

Congratulations! You've completed the LangGraph Core Concepts notebook. Let's recap what you've learned:

### What We Covered

1. **State - The Foundation**
   - Shared data structure that flows through your workflow
   - Defined using TypedDict for type safety
   - Like tracking information during SCM address object creation (name, IP, folder, validation status)
   - Each node can read and update state
   - Progressive complexity levels: Simple (2-3 fields) → Advanced (10+ fields)

2. **Nodes - Processing Units**
   - Individual functions that perform specific tasks
   - Like stages in a configuration workflow (parse, validate, format)
   - Receive state as input, return partial state updates
   - Each node has a single, clear responsibility

3. **StateGraph - The Builder**
   - Builds and compiles your graph structure
   - Manages nodes, edges, and state flow
   - Like a configuration playbook template that defines the entire procedure
   - First component you interact with when building workflows

4. **Edges - Connections**
   - Link nodes together to define workflow sequence
   - Like API call sequences in configuration workflows
   - Determine which node executes next
   - Simple edges follow a fixed path (START → node1 → node2 → END)

5. **START and END - Entry and Exit Points**
   - START: Virtual entry point where workflow begins
   - END: Exit point where workflow completes
   - Not real nodes, just markers for execution boundaries

6. **Conditional Routing - Preview**
   - Make routing decisions based on state (covered in Notebook 103)
   - Like if/else logic or network ACL rules
   - Enable workflows to handle success/failure paths differently
   - Coming in Notebook 103: LLM-powered conditional routing

7. **Tools - Preview**
   - Reusable operations for external capabilities (covered in Notebook 106)
   - Like pan-scm-sdk methods: create(), list(), fetch(), update(), delete()
   - Different from nodes: tools do work, nodes orchestrate workflow
   - Coming in Notebook 106: ReAct agents that intelligently select and use tools

8. **Runnable - Execution Interface**
   - Standardized executable component
   - What you get when you compile a StateGraph
   - Like modular rack-mounted equipment - standard interface, swappable
   - Invoke it with initial state to execute your workflow

9. **Messages - LLM Communication**
   - Five types: Human, AI, System, Tool, Function
   - Used in conversational/agent-based workflows with LLMs
   - Not needed for simple automation, essential for AI agents
   - Coming in Notebook 103: Using Messages for conversational workflows

10. **Hands-On Exercises**
    - Exercise 1: Build a 3-node address object pipeline
    - Exercise 2: Design security policy state schema
    - Exercise 3: Add error handling
    - Bonus: Progressive complexity levels (1-4)

### Why This Matters for Network Automation

These core concepts are the foundation for building intelligent network automation workflows:
- **State** tracks your SCM configurations, validation results, creation progress
- **Nodes** represent automation tasks (parse input, validate data, format for API)
- **Edges** define your automation sequence
- **Conditional Edges** (Notebook 103) handle different scenarios (success vs failure)
- **Tools** (Notebook 106) interact with your infrastructure (SCM API calls)

### What You've Built

In this notebook, you created:
- ✅ Complete SCM address object creation workflow (validate → prepare → create → verify)
- ✅ Simple 3-node pipeline (parse → validate → format)
- ✅ State schemas for address objects with progressive complexity
- ✅ Graph visualizations showing workflow structure
- ✅ Error handling and validation patterns

### Next Steps

Now that you understand LangGraph fundamentals, you're ready to:
1. **Notebook 103**: Add conditional routing and LLM integration for intelligent decision-making
2. **Notebook 106**: Integrate real SCM API calls using pan-scm-sdk as tools
3. Build more complex workflows with multiple conditional paths
4. Implement error handling and retry logic
5. Create production-ready automation workflows

### Key Reminder

You don't need to memorize every detail - just understand how these components work together:
- **State** flows through **Nodes** connected by **Edges**
- **StateGraph** builds the workflow, compiles to a **Runnable**
- **Tools** provide reusable operations (coming in Notebook 106)
- **Messages** enable LLM interaction (coming in Notebook 103)

The best way to learn is by building! Try the hands-on exercises, then move on to Notebook 103 where you'll add conditional routing and LLM-powered decision making.

Great work! 🎉